<a href="https://colab.research.google.com/github/FacuML/NLP/blob/main/001_Preprocesamiento_Avanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuaderno Clase PLN - Preprocesamiento Avanzado

**Objetivos:**
*   Repasar la limpieza básica de texto.
*   Entender y aplicar Stemming (NLTK).
*   Entender y aplicar Lematización (spaCy).
*   Comparar los resultados de ambas técnicas.
*   Reflexionar sobre el impacto del preprocesamiento.

**Agenda:**

1.  Instalaciones e Importaciones
2.  Repaso: Limpieza básica y Tokenización
3.  El problema de las variantes de palabras
4.  Stemming con NLTK
5.  Lematización con spaCy
6.  Comparación Stemming vs. Lematización
7.  Micro-Laboratorio (Ejercicio Práctico)
8.  Brainstorming

# 1. Instalaciones e Importaciones

In [ ]:
# Instalar librerías (si no están ya en Colab)
!pip install nltk spacy > /dev/null
!python -m spacy download es_core_news_sm > /dev/null # Modelo pequeño de español para spaCy

In [ ]:
# Importar librerías
import nltk
import spacy
import re # Para expresiones regulares (limpieza)
import string # Para signos de puntuación

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Cargar modelo de spaCy en español
nlp = spacy.load('es_core_news_sm')
print("Modelo de spaCy 'es_core_news_sm' cargado.")

Modelo de spaCy 'es_core_news_sm' cargado.


In [ ]:
# Cargar stopwords en español de NLTK
stopwords_es = nltk.corpus.stopwords.words('spanish')

In [ ]:
# Añadir algunas stopwords comunes si es necesario (opcional)
# stopwords_es.extend(['tan', 'van', 'ser', 'haber', 'ir'])
print(f"\nEjemplo de stopwords en español (NLTK): {stopwords_es[:15]}...")


Ejemplo de stopwords en español (NLTK): ['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con']...


# 2. Repaso: Limpieza básica y Tokenización

Recordemos los pasos comunes que ya vimos:
*   **Pasar a minúsculas:** Para tratar "Hola" y "hola" igual.
*   **Quitar números:** A menudo no aportan significado general.
*   **Quitar signos de puntuación:** Como ',', '.', '!', '?'.
*   **Quitar stopwords:** Palabras muy comunes ("el", "la", "de", "que", "y"...) que aparecen mucho pero no suelen distinguir el tema del texto.
*   **Tokenización:** Dividir el texto en unidades (palabras o "tokens").

In [ ]:
# Ejemplo de texto
texto_ejemplo = "Los niños corrían rápidamente por el parque, jugando y riendo. ¡Qué día más lindo!"

In [ ]:
# Función simple de limpieza y tokenización (usando NLTK para stopwords y tokenización)
def limpiar_tokenizar_basico(texto):
  # 1. Minúsculas
  texto = texto.lower()
  # 2. Quitar números (usando expresiones regulares)
  texto = re.sub(r'\d+', '', texto)
  # 3. Quitar puntuación
  texto = texto.translate(str.maketrans('', '', string.punctuation + '¡¿'))
  # 4. Quitar espacios extra
  texto = texto.strip()
  # 5. Tokenizar
  tokens = nltk.word_tokenize(texto, language='spanish')
  # 6. Quitar stopwords
  tokens_limpios = [palabra for palabra in tokens if palabra not in stopwords_es]
  return tokens_limpios

In [ ]:
# Aplicar la función
tokens_basicos = limpiar_tokenizar_basico(texto_ejemplo)
print("Texto original:", texto_ejemplo)
print("Tokens después de limpieza básica y quitar stopwords (NLTK):", tokens_basicos)

Texto original: Los niños corrían rápidamente por el parque, jugando y riendo. ¡Qué día más lindo!
Tokens después de limpieza básica y quitar stopwords (NLTK): ['niños', 'corrían', 'rápidamente', 'parque', 'jugando', 'riendo', 'día', 'lindo']


# 3. El problema de las variantes de palabras

Observen los tokens resultantes: `['niños', 'corrían', 'rápidamente', 'parque', 'jugando', 'riendo', 'día', 'lindo']`.

Tenemos "corrían", "jugando", "riendo". Si tuviéramos otro texto con "correr", "juega", "reír", serían tokens diferentes.

**¿No sería útil agrupar las palabras que comparten una raíz o significado base?**

*   **corrían, correr, corremos, corrió -> CORRER**
*   **jugando, juega, jugamos -> JUGAR**

Esto ayuda a:
*   Reducir el tamaño del vocabulario (menos columnas en BoW/TF-IDF).
*   Agrupar conceptos similares.

Dos técnicas principales para esto: **Stemming** y **Lematización**.

# 4. Stemming con NLTK

*   **¿Qué es?** Un proceso **heurístico** (basado en reglas simples) para cortar el final de las palabras y obtener su "raíz" o "stem".
*   **No siempre produce una palabra real** del diccionario.
*   **Ventajas:** Rápido, simple, reduce mucho el vocabulario.
*   **Desventajas:** A veces "corta" demasiado o agrupa palabras incorrectamente. No considera el contexto gramatical.
*   **Herramienta:** NLTK tiene `SnowballStemmer` para español.

In [ ]:
from nltk.stem import SnowballStemmer

# Crear un stemmer para español
stemmer = SnowballStemmer('spanish')

In [ ]:
# Vamos a aplicar stemming a los tokens que obtuvimos antes (después de limpieza básica)
stems_nltk = [stemmer.stem(token) for token in tokens_basicos]

print("Tokens originales (limpios):", tokens_basicos)
print("Stems resultantes (NLTK): ", stems_nltk)

Tokens originales (limpios): ['niños', 'corrían', 'rápidamente', 'parque', 'jugando', 'riendo', 'día', 'lindo']
Stems resultantes (NLTK):  ['niñ', 'corr', 'rapid', 'parqu', 'jug', 'riend', 'dia', 'lind']


In [ ]:
# Probemos con otras palabras relacionadas
palabras_relacionadas = ['correr', 'corría', 'corriendo', 'corredor', 'corredores']
stems_relacionadas = [stemmer.stem(p) for p in palabras_relacionadas]
print(f"\nStemming para {palabras_relacionadas}: {stems_relacionadas}") # Notar que agrupa bien


Stemming para ['correr', 'corría', 'corriendo', 'corredor', 'corredores']: ['corr', 'corr', 'corr', 'corredor', 'corredor']


In [ ]:
palabras_problematicas = ['computadora', 'computación', 'computar']
stems_problematicos = [stemmer.stem(p) for p in palabras_problematicas]
print(f"Stemming para {palabras_problematicas}: {stems_problematicos}") # Funciona razonable

Stemming para ['computadora', 'computación', 'computar']: ['comput', 'comput', 'comput']


In [ ]:
palabras_problematicas_2 = ['universidad', 'universo']
stems_problematicos_2 = [stemmer.stem(p) for p in palabras_problematicas_2]
print(f"Stemming para {palabras_problematicas_2}: {stems_problematicos_2}") # ¡Ojo! Puede agrupar de más

Stemming para ['universidad', 'universo']: ['univers', 'univers']


# 5. Lematización con spaCy

*   **¿Qué es?** Un proceso más **lingüístico**, basado en diccionarios y análisis morfológico, para encontrar la forma canónica o de diccionario de una palabra (su "lema").
*   **Produce palabras reales**.
*   **Ventajas:** Más preciso conceptualmente, mejor para análisis semántico.
*   **Desventajas:** Más lento computacionalmente, requiere modelos lingüísticos (como los de spaCy).
*   **Herramienta:** spaCy lo hace automáticamente al procesar el texto con un modelo cargado (`nlp()`). El lema está en el atributo `token.lemma_`. spaCy también identifica stopwords (`token.is_stop`).

In [ ]:
# Usaremos spaCy directamente sobre el texto original limpio (sin quitar stopwords aún)
# porque spaCy necesita el contexto para lematizar bien.

def limpiar_texto_spacy(texto):
  # 1. Minúsculas
  texto = texto.lower()
  # 2. Quitar números
  texto = re.sub(r'\d+', '', texto)
  # 3. Quitar puntuación (dejamos espacios)
  texto = texto.translate(str.maketrans(string.punctuation + '¡¿', ' ' * len(string.punctuation + '¡¿')))
  # 4. Quitar espacios extra
  texto = re.sub(r'\s+', ' ', texto).strip()
  return texto

In [ ]:
texto_limpio_spacy = limpiar_texto_spacy(texto_ejemplo)
print("Texto limpio para spaCy:", texto_limpio_spacy)

Texto limpio para spaCy: los niños corrían rápidamente por el parque jugando y riendo qué día más lindo


In [ ]:
# Procesar el texto limpio con spaCy
doc = nlp(texto_limpio_spacy)

In [ ]:
# Obtener los lemas, filtrando stopwords y tokens no alfabéticos
lemas_spacy = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

print("\nLemas resultantes (spaCy, filtrando stopwords y no alfabéticos):", lemas_spacy)


Lemas resultantes (spaCy, filtrando stopwords y no alfabéticos): ['niño', 'correr', 'rápidamente', 'parque', 'jugar', 'reir', 'lindo']


In [ ]:
# Veamos los lemas de las palabras relacionadas
doc_relacionadas = nlp("correr corría corriendo corredor corredores")
lemas_relacionadas_spacy = [token.lemma_ for token in doc_relacionadas]
print(f"\nLemas para {' '.join([t.text for t in doc_relacionadas])}: {lemas_relacionadas_spacy}") # ¡Excelente! "corredor" es distinto de "correr"


Lemas para correr corría corriendo corredor corredores: ['correr', 'correr', 'correr', 'corredor', 'corredor']


In [ ]:
doc_problematicas = nlp("computadora computación computar")
lemas_problematicas_spacy = [token.lemma_ for token in doc_problematicas]
print(f"Lemas para {' '.join([t.text for t in doc_problematicas])}: {lemas_problematicas_spacy}") # "computación" se mantiene

Lemas para computadora computación computar: ['computadoro', 'computación', 'computar']


In [ ]:
doc_problematicas_2 = nlp("universidad universo")
lemas_problematicas_2_spacy = [token.lemma_ for token in doc_problematicas_2]
print(f"Lemas para {' '.join([t.text for t in doc_problematicas_2])}: {lemas_problematicas_2_spacy}") # Correctamente separados

Lemas para universidad universo: ['universidad', 'universo']


# 6. Comparación Stemming vs. Lematización

Veamos lado a lado los resultados para nuestro texto de ejemplo:

*   **Tokens originales (limpios):** `['niños', 'corrían', 'rápidamente', 'parque', 'jugando', 'riendo', 'día', 'lindo']`
*   **Stems (NLTK):** `['niñ', 'corr', 'rapid', 'parqu', 'jug', 'riend', 'dia', 'lind']`
*   **Lemas (spaCy):** `['niño', 'correr', 'rápidamente', 'parque', 'jugar', 'reír', 'día', 'lindo']`

**Observaciones:**
*   Los lemas son palabras reales, los stems no siempre.
*   La lematización parece capturar mejor la forma base ("correr", "jugar", "reír").
*   El stemming es más agresivo ("niñ", "corr", "rapid").
*   Ambos acortan "día" (sin tilde) de forma similar en este caso (NLTK por stem, spaCy porque el modelo puede no tener la tilde en su lema base).

**¿Cuándo usar cuál?**
*   **Stemming:** Cuando la velocidad es crucial y no importa tanto la interpretabilidad (ej: recuperación de información a gran escala).
*   **Lematización:** Cuando la precisión semántica y la interpretabilidad son importantes (ej: análisis de sentimiento, clasificación de temas, chatbots). **Generalmente preferido si los recursos computacionales lo permiten.**

# 7. Micro-Laboratorio (Ejercicio Práctico)

**Consigna:**

Dado el siguiente conjunto de frases (reviews de películas):
1.  Definir una función `preprocesar_nltk(texto)` que:
    *   Limpie el texto (minúsculas, números, puntuación).
    *   Tokenice.
    *   Quite stopwords (usando la lista de NLTK).
    *   Aplique Stemming (con `SnowballStemmer`).
    *   Devuelva la lista de stems.
2.  Definir una función `preprocesar_spacy(texto)` que:
    *   Limpie el texto (minúsculas, números, puntuación - cuidado con no quitar espacios necesarios para spaCy).
    *   Procese el texto con `nlp()`.
    *   Devuelva la lista de lemas de los tokens que no sean stopwords (`token.is_stop`) y sean alfabéticos (`token.is_alpha`).
3.  Aplicar ambas funciones a cada frase del dataset `reviews`.
4.  Imprimir los resultados de ambas funciones para cada frase, uno debajo del otro, para poder comparar.
5.  **Observar:** ¿Qué diferencias notables encuentran? ¿En qué casos un método parece funcionar "mejor" que el otro?

In [ ]:
# Dataset para el ejercicio
reviews = [
    "Una película emocionante con actuaciones brillantes. ¡Me encantó!",
    "Muy aburrida y lenta. El guión era predecible y los actores no convencían.",
    "Los efectos especiales fueron impresionantes, pero la historia dejaba mucho que desear.",
    "¡Qué gran comedia! Me reí sin parar durante toda la película.",
    "Un documental necesario que aborda temas importantes con profundidad y sensibilidad."
]

## .1 Procesar con NLTK las reviews (Steeming).


In [ ]:
def procesar_nltk(lista_reviews):
  stemmer = SnowballStemmer('spanish')
  all_stems_reviews=[]
  for review in lista_reviews:
    texto= review.lower()
    texto = re.sub(r'\d+', '', texto)
    texto = texto.translate(str.maketrans('', '', string.punctuation + '¡¿'))
    texto = texto.strip()
    tokens = nltk.word_tokenize(texto, language='spanish')
    tokens_limpios = [palabras for palabras in tokens if palabras not in stopwords_es]
    stems = [stemmer.stem(token)for token in tokens_limpios]
    all_stems_reviews.append(stems)
  return all_stems_reviews

In [ ]:
print(procesar_nltk(reviews))

[['pelicul', 'emocion', 'actuacion', 'brillant', 'encant'], ['aburr', 'lent', 'guion', 'predec', 'actor', 'convenc'], ['efect', 'especial', 'impresion', 'histori', 'dej', 'des'], ['gran', 'comedi', 'rei', 'par', 'tod', 'pelicul'], ['documental', 'necesari', 'abord', 'tem', 'import', 'profund', 'sensibil']]


## .2 Procesar con Spacy las reviews (Lematisation).

In [ ]:
def procesar_spacy(lista_reviews):
  all_stems_reviews=[]
  for review in lista_reviews:
    texto= review.lower()
    texto = re.sub(r'\d+', '', texto)
    texto = texto.translate(str.maketrans('', '', string.punctuation + '¡¿'))
    texto = texto.strip()
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    all_stems_reviews.append(lemas)
  return all_stems_reviews

In [ ]:
print(procesar_spacy(reviews))

[['película', 'emocionante', 'actuación', 'brillante', 'encantar'], ['aburrido', 'lento', 'guión', 'predecible', 'actor', 'convencer'], ['efecto', 'especial', 'impresionante', 'historia', 'dejar', 'desear'], ['comedia', 'reí', 'parar', 'película'], ['documental', 'necesario', 'abordar', 'tema', 'importante', 'profundidad', 'sensibilidad']]


## .4 Impresion y comparacion de ambas funciones

In [ ]:
print("Reviews originales:")
for review in reviews:
  print(review)
print("\nStems (NLTK):")
for stems in procesar_nltk(reviews):
  print(stems)
print("\nLemas (spaCy):")
for lemas in procesar_spacy(reviews):
  print(lemas)

Reviews originales:
Una película emocionante con actuaciones brillantes. ¡Me encantó!
Muy aburrida y lenta. El guión era predecible y los actores no convencían.
Los efectos especiales fueron impresionantes, pero la historia dejaba mucho que desear.
¡Qué gran comedia! Me reí sin parar durante toda la película.
Un documental necesario que aborda temas importantes con profundidad y sensibilidad.

Stems (NLTK):
['pelicul', 'emocion', 'actuacion', 'brillant', 'encant']
['aburr', 'lent', 'guion', 'predec', 'actor', 'convenc']
['efect', 'especial', 'impresion', 'histori', 'dej', 'des']
['gran', 'comedi', 'rei', 'par', 'tod', 'pelicul']
['documental', 'necesari', 'abord', 'tem', 'import', 'profund', 'sensibil']

Lemas (spaCy):
['película', 'emocionante', 'actuación', 'brillante', 'encantar']
['aburrido', 'lento', 'guión', 'predecible', 'actor', 'convencer']
['efecto', 'especial', 'impresionante', 'historia', 'dejar', 'desear']
['comedia', 'reí', 'parar', 'película']
['documental', 'necesario',

# 8. Brainstorming

Ahora que conocemos estas técnicas, pensemos:

**¿Cómo podemos preprocesar el texto de manera que se eviten sesgos y discriminaciones?**

*   ¿Qué pasa si la lista de `stopwords` que usamos (sea de NLTK o spaCy) quita palabras importantes para un grupo minoritario o en un contexto específico (ej: jerga, términos culturales)?
*   ¿Los stemmers o lematizadores funcionan igual de bien con diferentes dialectos del español o con lenguaje inclusivo? (Probablemente no, los modelos estándar están entrenados en textos más "formales").
*   Si quitamos nombres propios o entidades, ¿podríamos estar eliminando información crucial sobre representación?
*   Al elegir agresividad (stemming) vs. precisión (lematización), ¿podríamos afectar diferencialmente el análisis de textos de distintos grupos?
*   ¿Qué responsabilidad tenemos al elegir y aplicar estas técnicas? ¿Deberíamos documentar siempre nuestras decisiones de preprocesamiento?

**(Discusión en grupo)**